In [5]:
import typing
if not hasattr(typing, "NotRequired"):
    from typing_extensions import NotRequired
    typing.NotRequired = NotRequired
from mp_api.client import MPRester

# 1. 使用 MPRester 获取材料数据
mcw_api_key = "E3tJQOXbTZpaiDUYVOlW4kdm0NDR8X9J"

with MPRester(mcw_api_key) as mpr:
    # do stuff with mpr...
    docs = mpr.materials.summary.search(
        elements=["O"], 
        is_stable=True,
        fields=[
            "material_id",
            "formula_pretty",
            "structure",
            "band_gap",
            "energy_above_hull",
            "density",
            "bulk_modulus",   # 替代 k_voigt
            "shear_modulus",  # 替代 g_voigt
            "efermi",         # 费米能级 (Fermi Level)
            "is_metal"        # 是否为金属
        ]
    )
    
    # 打印结果示例
    for doc in docs[:3]:
        print(f"化学式: {doc.formula_pretty}")
        print(f"体积模量 (Bulk Modulus): {doc.bulk_modulus} GPa")
        print(f"剪切模量 (Shear Modulus): {doc.shear_modulus} GPa")
        print(f"带隙: {doc.band_gap} eV")
        print("-" * 30)

    # 3. 将结构保存为 CIF 文件 (用于你的几何生成模型)
    if docs:
        first_structure = docs[0].structure
        first_structure.to(filename="../example_crystal.cif")

Retrieving SummaryDoc documents: 100%|██████████| 10325/10325 [03:47<00:00, 45.33it/s]

化学式: FeO
体积模量 (Bulk Modulus): None GPa
剪切模量 (Shear Modulus): None GPa
带隙: 1.8157 eV
------------------------------
化学式: PbO
体积模量 (Bulk Modulus): {'voigt': 25.606, 'reuss': 5.662, 'vrh': 15.634} GPa
剪切模量 (Shear Modulus): {'voigt': 16.972, 'reuss': 4.072, 'vrh': 10.522} GPa
带隙: 1.4635999999999991 eV
------------------------------
化学式: SnO
体积模量 (Bulk Modulus): {'voigt': 40.401, 'reuss': 19.109, 'vrh': 29.755} GPa
剪切模量 (Shear Modulus): {'voigt': 30.353, 'reuss': 12.136, 'vrh': 21.245} GPa
带隙: 0.4096 eV
------------------------------


In [5]:
import typing
if not hasattr(typing, "NotRequired"):
    from typing_extensions import NotRequired
    typing.NotRequired = NotRequired
from mp_api.client import MPRester

API_KEY = "E3tJQOXbTZpaiDUYVOlW4kdm0NDR8X9J"

with MPRester(API_KEY) as mpr:
    # 获取 mpr 对象的所有属性
    attrs = dir(mpr)
    
    # 1. 寻找压电 (Piezo) 相关的属性名
    piezo_attr = [a for a in attrs if 'piezo' in a.lower()]
    print(f"探测到的压电属性名: {piezo_attr}")
    
    # 2. 寻找 DOS 相关的属性名
    dos_attr = [a for a in attrs if 'dos' in a.lower() or 'electronic' in a.lower()]
    print(f"探测到的 DOS/电子结构属性名: {dos_attr}")

    # 3. 验证探测到的第一个路径是否可用 (以第一个找到的为准)
    if piezo_attr:
        try:
            # 动态获取属性并搜索
            p_rester = getattr(mpr, piezo_attr[0])
            test_p = p_rester.search(material_ids=["mp-22862"], fields=["material_id"])
            print(f"✅ 验证成功！压电路径应使用: mpr.{piezo_attr[0]}")
        except Exception as e:
            print(f"❌ 验证压电路径失败: {e}")

    if dos_attr:
        try:
            # 动态获取属性并搜索
            d_rester = getattr(mpr, dos_attr[0])
            test_d = d_rester.search(material_ids=["mp-22862"], fields=["material_id"])
            print(f"✅ 验证成功！DOS 路径应使用: mpr.{dos_attr[0]}")
        except Exception as e:
            print(f"❌ 验证 DOS 路径失败: {e}")


with MPRester(API_KEY) as mpr:
    # --- 查看压电路由的所有可用字段 ---
    print("--- 压电 (Piezoelectric) 可用字段 ---")
    # 注意：在最新版 mp-api 中，使用 .available_fields
    piezo_fields = mpr.materials.piezoelectric.available_fields
    print(piezo_fields)
    
    print("\n" + "="*50 + "\n")
    
    # --- 查看电子结构/DOS 路由的所有可用字段 ---
    print("--- 电子结构 (Electronic Structure) 可用字段 ---")
    dos_fields = mpr.materials.electronic_structure.available_fields
    print(dos_fields)

探测到的压电属性名: ['piezoelectric']
探测到的 DOS/电子结构属性名: ['electronic_structure', 'electronic_structure_bandstructure', 'electronic_structure_dos', 'get_dos_by_material_id', 'get_phonon_dos_by_material_id']


/tmp/ipykernel_6196/3440346513.py:25: DeprecationWarning: Accessing piezoelectric data through MPRester.piezoelectric is deprecated. Please use MPRester.materials.piezoelectric instead.
  p_rester = getattr(mpr, piezo_attr[0])
Retrieving PiezoelectricDoc documents: 0it [00:00, ?it/s]
/tmp/ipykernel_6196/3440346513.py:34: DeprecationWarning: Accessing electronic_structure data through MPRester.electronic_structure is deprecated. Please use MPRester.materials.electronic_structure instead.
  d_rester = getattr(mpr, dos_attr[0])


✅ 验证成功！压电路径应使用: mpr.piezoelectric


Retrieving ElectronicStructureDoc documents: 100%|██████████| 1/1 [00:00<00:00, 3934.62it/s]

✅ 验证成功！DOS 路径应使用: mpr.electronic_structure
--- 压电 (Piezoelectric) 可用字段 ---
['builder_meta', 'nsites', 'elements', 'nelements', 'composition', 'composition_reduced', 'formula_pretty', 'formula_anonymous', 'chemsys', 'volume', 'density', 'density_atomic', 'symmetry', 'material_id', 'deprecated', 'deprecation_reasons', 'last_updated', 'origins', 'warnings', 'property_name', 'total', 'ionic', 'electronic', 'e_ij_max', 'max_direction', 'strain_for_max']


--- 电子结构 (Electronic Structure) 可用字段 ---
['task_id', 'band_gap', 'cbm', 'vbm', 'efermi', 'is_gap_direct', 'is_metal', 'magnetic_ordering', 'builder_meta', 'nsites', 'elements', 'nelements', 'composition', 'composition_reduced', 'formula_pretty', 'formula_anonymous', 'chemsys', 'volume', 'density', 'density_atomic', 'symmetry', 'material_id', 'deprecated', 'deprecation_reasons', 'last_updated', 'origins', 'warnings', 'property_name', 'bandstructure', 'dos']
